# LDA

In [13]:
import json
import numpy as np
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import Normalizer

In [2]:
class News:
    def __init__(self, id, date, title, content, url, siteType):
        self.id = id
        self.date = date
        self.title = title
        self.content = content
        self.url = url
        self.siteType = siteType
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

In [3]:
news = []
with open('/data/kasandra/year/10k.test.normalized.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

In [4]:
words = []

sub_news = news

for n in sub_news:
    words.extend(n.content.split())
counts = Counter(words)
one_time = [k for k, v in dict(counts).items() if v == 1]
print("total words: %s" % (len(words) - len(one_time)))

news_content = [x.content for x in sub_news]

total words: 2717122


In [5]:
def print_scores(tfidf_matrix):
    print("Количество слов после векторизации: %s" % tfidf_matrix.shape[1])

    print("Максимальное значение вектота: %s" % tfidf_matrix.max())
    print("Минимальное значение вектора: %s" % tfidf_matrix.min())

    try:
        lengths = [np.linalg.norm(t.toarray()) for t in tfidf_matrix]
    except BaseException:
        lengths = [np.linalg.norm(t) for t in tfidf_matrix]

    print("Максимальная длинна вектора: %s" % max(lengths))
    print("Минимальная длинна вектора: %s" % min(lengths))
    print("Средняя длинна вектора: %s" % np.mean(lengths))

# TfIdf

In [6]:
tfidf_vectorizer = TfidfVectorizer(norm='l2', use_idf=True, tokenizer=lambda text: text.split(" "))
tfidf_matrix = tfidf_vectorizer.fit_transform(news_content)

In [7]:
print_scores(tfidf_matrix)

Количество слов после векторизации: 66802
Максимальное значение вектота: 0.869933483272
Минимальное значение вектора: 0.0
Максимальная длинна вектора: 1.0
Минимальная длинна вектора: 1.0
Средняя длинна вектора: 1.0


In [8]:
# tf = CountVectorizer(stop_words=stopwords).fit_transform(content)
lda = LatentDirichletAllocation(n_topics=20, max_iter=10, learning_method='online', learning_offset=50.)
lda_matrix = lda.fit_transform(tfidf_matrix)

In [9]:
print_scores(lda_matrix)

Количество слов после векторизации: 20
Максимальное значение вектота: 0.970074814217
Минимальное значение вектора: 0.00157500690319
Максимальная длинна вектора: 0.970099107122
Минимальная длинна вектора: 0.643810075345
Средняя длинна вектора: 0.904774734509


In [10]:
normalizer = Normalizer()
norm_matrix = normalizer.fit_transform(lda_matrix)

In [11]:
print_scores(norm_matrix)

Количество слов после векторизации: 20
Максимальное значение вектота: 0.999974958327
Минимальное значение вектора: 0.00162355257482
Максимальная длинна вектора: 1.0
Минимальная длинна вектора: 1.0
Средняя длинна вектора: 1.0


# Tf

In [17]:
tf = CountVectorizer().fit_transform(news_content)

In [18]:
lda = LatentDirichletAllocation(n_topics=20, max_iter=10, learning_method='online', learning_offset=50.)
lda_matrix = lda.fit_transform(tf)

In [19]:
print_scores(lda_matrix)

Количество слов после векторизации: 20
Максимальное значение вектота: 0.999256068907
Минимальное значение вектора: 4.26657564647e-06
Максимальная длинна вектора: 0.999256083482
Минимальная длинна вектора: 0.365900834622
Средняя длинна вектора: 0.726858737212


In [20]:
normalizer = Normalizer()
norm_matrix = normalizer.fit_transform(lda_matrix)

In [21]:
print_scores(norm_matrix)

Количество слов после векторизации: 20
Максимальное значение вектота: 0.999999985414
Минимальное значение вектора: 6.86399007595e-06
Максимальная длинна вектора: 1.0
Минимальная длинна вектора: 1.0
Средняя длинна вектора: 1.0
